In [2]:
from miditok import REMI
from pathlib import Path

# Creates the tokenizer and list the file paths
tokenizer = REMI(sos_eos=True)
midi_paths = list(Path('Final_Project/big_dataset').glob('**/*.midi'))

# A validation method to discard MIDIs we do not want
def midi_valid(midi) -> bool:
    if any(ts.numerator != 4 for ts in midi.time_signature_changes):
        return False  # time signature different from 4/*, 4 beats per bar
    if midi.max_tick < 10 * midi.ticks_per_beat:
        return False  # this MIDI is too short
    return True

# Converts MIDI files to tokens saved as JSON files
tokenizer.tokenize_midi_dataset(        
    midi_paths,
    Path('Final_Project/tokens_noBPE'),
    midi_valid,
)

# Learns the vocabulary with BPE
tokenizer.learn_bpe(
    vocab_size=512,
    tokens_paths=list(Path('Final_Project/tokens_noBPE').glob("**/*.json")),
    start_from_empty_voc=False
)

# Converts the tokenized musics into tokens with BPE
tokenizer.apply_bpe_to_dataset(Path('Final_Project/tokens_noBPE'), Path('Final_Project/tokens_BPE'))

tokenizer.learn_bpe()

Tokenizing MIDIs (Final_Project/tokens_noBPE): 100%|██████████| 1276/1276 [12:46<00:00,  1.67it/s]


TypeError: MIDITokenizer.learn_bpe() got an unexpected keyword argument 'tokens_paths'

In [9]:
path_nobpe = 'Final_Project/tokens_noBPE'
path_bpe = 'Final_Project/tokens_BPE'

tokenizer.learn_bpe(
    path_nobpe,
    512,
    path_bpe,
)

# Converts the tokenized musics into tokens with BPE
tokenizer.apply_bpe_to_dataset(Path('Final_Project/tokens_noBPE'), Path('Final_Project/tokens_BPE'))

Learning byte pair encoding: 100%|██████████| 292/292 [1:53:18<00:00, 23.28s/it, seq_len_variation=-30.26, avg_nb_token_combs=2.51, max_nb_token_combs=3]


Mean of original lengths: 20069.75313479624
Mean length after BPE: 13997.225705329154
Variation from original: -30.26 %


Applying BPE to dataset: 100%|██████████| 1276/1276 [22:35<00:00,  1.06s/it] 


In [3]:
from miditok.utils import get_midi_programs

In [6]:
import json
import numpy as np
import os
filenames = 'Final_Project/tokens_BPE/'
# Turns all the json files in a folder into numpy arrays
def json_to_nparray():
    for filename in os.listdir(filenames):
        if filename.endswith('.json'):
            with open(filenames + filename) as f:
                data = json.load(f)
                np.save(filenames + filename[:-5], np.array(data))
            continue
        else:
            continue
json_to_nparray()

In [8]:
hure = np.load(('Final_Project/tokens_BPE/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--1.npy'), allow_pickle=True)[()].get('ids')

In [13]:
len(hure)

1

In [14]:
from miditoolkit import MidiFile
midi = MidiFile('Final_Project/big_dataset/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--1.midi')
coverted = tokenizer(hure,[0, False])


KeyError: 230

In [2]:
tokenizer.save_params(Path('Final_Project/tokens_BPE/config.txt'))